<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_0903BERT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import platform
isColab = True if platform.system() == 'Linux' else False

if isColab:
    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1 
    !pip install mecab-python3 > /dev/null 2>&1 
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1 
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1 

    import subprocess
    cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,shell=True).communicate()[0]).decode('utf-8')

    !pip install fugashi[unidic] > /dev/null 2>&1 
    !python -m unidic download > /dev/null 2>&1 
    !pip install ipadic > /dev/null 2>&1
    
    # huggingface の transformers をインストール
    !pip  instal transformers > /dev/null 2>&1 

# ごく簡単な BERT の使い方

- date: 2021_0903
- author: 浅川伸一
- 参考サイト: https://huggingface.co/bert-base-multilingual-cased


In [ ]:
# MeCab, fugashi, ipadic のインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

!pip install fugashi[unidic]
!python -m unidic download
!pip install ipadic

In [3]:
# allennlp の transformers のインストール
#!pip install transformers
from transformers import BertTokenizer, BertModel
from transformers import pipeline

In [4]:
# 学習済 BERT 日本語モデルのインストール special thanks to 東北大学 乾研究室
unmasker = pipeline('fill-mask', model='cl-tohoku/bert-base-japanese-whole-word-masking')

2022-09-29 07:00:55.096411: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [5]:
# ここからは，実演です
masked_sentences = [
                    ['本日 は [MASK] なり'],
                    #['ニューラルネットワーク は 神経 心理学 の [MASK] です.'],
                    # ['札幌 クラーク 病院 は 神経 心理学 の [MASK] です.'],
                    # ['東京女子大学 は [MASK] な 大学 です.'],
                    # ['[MASK] 大学 は リベラル な 大学 です.'],
                    ]
for sentence in masked_sentences:
    for x in unmasker(sentence):
        print(f"{x['sequence']}, スコア: {x['score']:.3f} ")

#for x in unmasker("ニューラルネットワーク は 神経 心理学 の [MASK] です."):
#    print(f"{x['sequence']}, スコア:{x['score']:.3f} ")
#print(unmasker("札幌 クラーク 病院 は 神経 心理学 の [MASK] です."))

本日 は と なり, スコア: 0.751 
本日 は 大相撲 なり, スコア: 0.037 
本日 は 開催 なり, スコア: 0.026 
本日 は 廃止 なり, スコア: 0.018 
本日 は に なり, スコア: 0.013 


In [8]:
print(tokenizer.encode('本日は[MASK]なり.'))
print(tokenizer.encode('本日 は [MASK] なり.'))

[2, 108, 32, 9, 4, 297, 143, 3]
[2, 108, 32, 9, 4, 297, 143, 3]


In [ ]:
tokenize
#unmasker(['CCAP は 神経 心理学 に とって [MASK] な 存在 です.'])

In [ ]:
# 以下は BERT モデルの内部を確認する作業
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
#print(model)

In [ ]:
from transformers import BertConfig

# 東北大学 乾研究室の設定を確認
config_japanese = BertConfig.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
print(config_japanese)

In [7]:
# 分かち書きをするための tokenizer を呼び出して利用するため
import torch 
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
input_ids = tokenizer.encode(f'''
    青葉山で{tokenizer.mask_token}の研究をしています。
''', return_tensors='pt')
print(input_ids)

In [ ]:
＃help(tokenizer)

In [ ]:
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.mask_token, tokenizer.mask_token_id)
print(tokenizer.bos_token, tokenizer.bos_token_id)
#print(tokenizer.eos_token, tokenizer.eos_token_id)


In [ ]:
print(tokenizer.encode('本日は晴天なり.', return_tensors='pt'))                # 平文，分かち書きしない場合 -> 独自の構文解析
print(tokenizer.encode(['本日','は','晴天','なり', '.'], return_tensors='pt')) # 文字列で構成されたリストを渡した場合 -> 構文解析はしない
print(tokenizer.encode('本日 は 晴天 なり .', return_tensors='pt'))            # 空白で分かち書きを区切った場合 -> 分かち書きを信用しない
print(tokenizer.encode('本日 は 晴天 なり', return_tensors='pt'))              # 文末にピリオド
#print(tokenizer.encode('私は誰', return_tensors='pt'))
#print(tokenizer.encode('本日は曇天なり'))
#inputs = tokenizer('無職転生と鬼滅の刃ではどちらが人気があるのだろうか？', '次の文章をセンテンスbとする', padding='max_length', return_tensors='pt')
#inputs = tokenizer('無職転生と鬼滅の刃ではどちらが人気があるのだろうか？', '次の文章をセンテンスbとする')
#print(inputs.keys())
#for k in ['input_ids', 'token_type_ids', 'attention_mask']:
#    print(f'key:{k} -> inputs[k]:{inputs[k]}')

[2, 108, 32, 9, 4, 297, 143, 3]

In [ ]:
# 分かち書きしたものを，もとに戻す確認作業
for id in  [   2,  108,   32,    9, 4798,  372,  297,  143,    3]:
    print(id, tokenizer.ids_to_tokens[id])

In [ ]:
# 「めかぶ」との違いを明確にするために，めかぶの解析結果を示します。
import MeCab
print(MeCab.Tagger().parse('本日は曇天なり'))

#help(tokenizer.encode)
#type(inputs)
#inputs.input_ids

In [ ]:
# 日本語
model

In [ ]:
#model.pooler.dense
model.encoder.layer[0]

<center>
<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/07/transformer.png"><br/>
トランスフォーマーモデル (Source: https://arxiv.org/abs/1706.03762)
</center>

In [ ]:
#tokenizer.ids_token
print(tokenizer.encode('今宵は月夜です', return_tensors='pt'))